# Get fiat deposits and withdrawals
### Only can handle 2 calls per minutes

Endpoint /sapi/v1/fiat/orders has weight of 90000, 180000 available each minutes

Run until limit variable is reached

In [ ]:
import asyncio

from datetime import datetime, timedelta
from analyst.settings import get_settings
from analyst.adapters.factory import get_adapters

In [ ]:
settings = get_settings()
adapters = await get_adapters(settings=settings)

limit = datetime(2018, 1, 1)

async with await adapters.binance.get_session() as session:
    end_time = datetime.now()
    delta = timedelta(days=0)
    
    while True:
        for transaction_type in range(2):
            params = {
                "transactionType": transaction_type,
                "beginTime": (end_time - timedelta(days=90)).strftime("%s000"),
                "endTime": end_time.strftime("%s000"),
                "timestamp": datetime.now().strftime("%s000")
            }
            params["signature"] = adapters.binance._get_signature(params)

            response = await session.get(
                f"{settings.binance.api_url}/sapi/v1/fiat/orders",
                params=params
            )

            orders = [
                order
                for order in (await response.json())["data"]
                if order["status"] == "Successful"
            ]

            for order in orders:
                print(
                    f"{'depot' if transaction_type == 0 else 'retrait':9s}",
                    datetime.fromtimestamp(order["updateTime"] / 1000).strftime("%x"),
                    f"amount = {order['amount']} total = {order['indicatedAmount']} fee = {order['totalFee']}"
                )

            await asyncio.sleep(30)
            
            break
        
        break

        end_time -= timedelta(days=90)
        
        if end_time < limit:
            break